# 1. 패키지 설치

In [1]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-chroma

Note: you may need to restart the kernel to use updated packages.


# 2. Knowledge Base 구성을 위한 데이터 생성

- [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)를 활용한 데이터 chunking
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 [TextSplitter](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)들을 제공
- `chunk_size` 는 split 된 chunk의 최대 크기
- `chunk_overlap`은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [92]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [93]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [3]:
from langchain_pinecone import PineconeVectorStore

# 데이터를 처음 저장할 때
index_name = 'table-markdown-index'

# Split documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_documents = text_splitter.split_documents(document_list)
print(f"Chunked documents length: {len(chunked_documents)}")

# Initialize the PineconeVectorStore
database = PineconeVectorStore.from_documents(
    documents=[],  # Start with an empty list
    embedding=embedding,
    index_name=index_name
)

# Upload documents in batches
batch_size = 100
for i in range(0, len(chunked_documents), batch_size):
    print(f'index: {i}, batch size: {batch_size}')
    batch = chunked_documents[i:i + batch_size]
    database.add_documents(batch)  # Add documents to the existing database

d:\inflearn-rag-notebook-main\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Chunked documents length: 451
index: 0, batch size: 100
index: 100, batch size: 100
index: 200, batch size: 100
index: 300, batch size: 100
index: 400, batch size: 100


# 3. 답변 생성을 위한 Retrieval

- `Chroma`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [110]:
query = '연봉 5천만원인 직장인의 종합소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [111]:
retrieved_docs

[Document(id='d4d0cfde-cd96-4458-b3cf-901c5d62dbf4', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [112]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [113]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

d:\inflearn-rag-notebook-main\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [114]:
retriever = database.as_retriever(search_kwargs={"k": 4})
retriever.invoke(query)

[Document(id='d4d0cfde-cd96-4458-b3cf-901c5d62dbf4', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [115]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [116]:
#query -> 직장인 -> 거주자로 바꾸는 체인
ai_message = qa_chain({"query": query})

In [117]:
# 강의에서는 위처럼 진행하지만 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [118]:
ai_message

{'query': '연봉 5천만원인 직장인의 종합소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 종합소득세는 526만원입니다. 이는 과세표준 5,000만원에 해당하는 세율 24%를 적용한 세액에서 근로소득세액공제를 차감하여 계산한 금액입니다.'}

In [153]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 반환해주세요.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [146]:
dictionary_chain

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template="\n    사용자의 질문을 보고, 우리의 ['사람을 나타내는 표현 -> 거주자']을(를) 참고해서 사용자의 질문을 변경해주세요.\n    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.\n    그런 경우에는 질문만 리턴해주세요.\n    연봉이 5천만원인 경우 624만원입니다.\n    사전: ['사람을 나타내는 표현 -> 거주자']\n\n    질문: {question}\n"), additional_kwargs={})])
| ChatUpstage(client=<openai.resources.chat.completions.completions.Completions object at 0x00000259DCF2D460>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000259DCE8F920>, model_kwargs={}, upstage_api_key=SecretStr('**********'), upstage_api_base='https://api.upstage.ai/v1/solar')
| StrOutputParser()

In [154]:
new_question = dictionary_chain.invoke({"question": query})

In [136]:
new_question

'질문: 거주자의 연봉이 5천만원인 경우, 종합소득세는 얼마인가요?'

In [155]:
# qa_chain.invoke()는 {"question": query}를 받음
# d
tax_chain = {"query": dictionary_chain} | qa_chain # dictionary_chain의 결과를 qa_chain에 넘겨줌

In [156]:
# tax_chain은 dictionary_chain이 먼저 돌아야함
# dictionary_chain은 {"question": query} 를 받기때문에 아래처럼 넣어줌
ai_response = tax_chain.invoke({"question": query})

In [157]:
ai_response

{'query': '질문: 연봉 5천만원인 거주자의 종합소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 종합소득세는 550만원입니다. 이는 과세표준 5천만원에 해당하는 세율 24%를 적용하고, 누진공제 624만원을 차감한 금액입니다.'}